---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [16]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()
# %matplotlib inline
# G = P1_Graphs[4]
# degrees = dict(G.degree())
# degree_values = sorted(set(degrees.values()))
# histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]

# plt.figure()
# plt.plot(degree_values,histogram, 'o')
# plt.xlabel('Degree')
# plt.ylabel('Fraction of Nodes')
# plt.xscale('log')
# plt.yscale('log')
# plt.show()

In [17]:

# print('average shortest path length: {}'.format(nx.average_shortest_path_length(G)))
# print('average clustering coefficient: {}'.format(nx.average_clustering(G)))
# plt.figure(figsize=(14, 8))
# nx.draw_networkx(G, pos=nx.circular_layout(G), with_labels=False, alpha=0.5)

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    return (['PA','SW_L','SW_L','PA','SW_H'])
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [41]:
def salary_predictions():
    
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.neural_network import MLPClassifier
    
    deg = nx.degree_centrality(G)
    close = nx.closeness_centrality(G,normalized = True)
    bet = nx.betweenness_centrality(G,normalized = True,endpoints = False)
    df = pd.DataFrame(G.nodes(data = True))
    
    df['deg'] = deg
    df['close'] = close
    df['bet'] = bet
    df['Department'] = df[1].map(lambda x : x['Department'])
    df['ManagementSalary'] = df[1].map(lambda x : x['ManagementSalary'])
    del df[1]
    df.rename(columns ={0 : 'nodes'},inplace =True )
    df.index = df['nodes']
    del df['nodes']
    train_data = df.dropna()
    df.fillna(value = 5,inplace = True)
    test_data = df[df['ManagementSalary']== 5]
    del test_data['ManagementSalary']
    
    features = ['deg','close','bet','Department']
    X_train = train_data[features]
    Y_train = train_data['ManagementSalary']
    X_test = test_data[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    return pd.Series(test_proba,X_test.index)
salary_predictions()

nodes
1       0.260164
2       0.402143
5       0.431939
8       0.345318
14      0.272794
18      0.251934
27      0.313105
30      0.311468
31      0.310923
34      0.309292
37      0.307665
40      0.306044
45      0.346139
54      0.378446
55      0.377848
60      0.268993
62      0.456374
65      0.247850
77      0.236283
79      0.235367
97      0.309860
101     0.307690
103     0.248053
108     0.251773
113     0.261773
122     0.200174
141     0.273345
142     0.434373
144     0.456354
145     0.424529
          ...   
913     0.050219
914     0.048571
915     0.045539
918     0.037487
923     0.041966
926     0.056781
931     0.056104
934     0.090337
939     0.036743
944     0.061478
945     0.039774
947     0.037179
950     0.089672
951     0.045808
953     0.041520
959     0.032853
962     0.038157
963     0.051950
968     0.045344
969     0.045235
974     0.032677
984     0.080423
987     0.053786
989     0.034612
991     0.072287
992     0.030290
994     0.051280
996     

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [5]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [14]:
def new_connections_predictions():
    
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.neural_network import MLPClassifier

    jaccard = list(nx.jaccard_coefficient(G))
    RA = list(nx.resource_allocation_index(G))
    PA = list(nx.preferential_attachment(G))

    df = pd.DataFrame(index = [(x[0],x[1]) for x in PA])
    df['PA'] = [x[2] for x in PA]
    df['RA'] = [x[2] for x in RA]
    df['jaccard'] = [x[2] for x in jaccard]
    for node in G.nodes():
            G.node[node]['community'] = G.node[node]['Department']
    CCN = list(nx.cn_soundarajan_hopcroft(G))
    df1 = pd.DataFrame(index = [(x[0],x[1]) for x in CCN])
    df1['CCN'] = [x[2] for x in CCN]
    df = pd.merge(df,df1,how = 'outer',left_index = True,right_index = True)
    df = pd.merge(df,future_connections,how = 'outer',left_index = True,right_index = True)


    train_data = df.dropna()
    df.fillna(value = 5,inplace = True)
    test_data = df[df['Future Connection']== 5]
    del test_data['Future Connection']
    
    features = ['PA','RA','jaccard','CCN']
    X_train = train_data[features]
    Y_train = train_data['Future Connection']
    X_test = test_data[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    return pd.Series(test_proba,X_test.index)
new_connections_predictions()

(0, 9)          0.027715
(0, 19)         0.062835
(0, 20)         0.278983
(0, 35)         0.011889
(0, 38)         0.007419
(0, 42)         0.808112
(0, 43)         0.019042
(0, 50)         0.009794
(0, 53)         0.554269
(0, 54)         0.043579
(0, 62)         0.315330
(0, 63)         0.035876
(0, 69)         0.020166
(0, 72)         0.010644
(0, 83)         0.159627
(0, 90)         0.029753
(0, 91)         0.015714
(0, 95)         0.135873
(0, 100)        0.027321
(0, 106)        0.952891
(0, 108)        0.013922
(0, 109)        0.010943
(0, 110)        0.010353
(0, 118)        0.008525
(0, 122)        0.030905
(0, 131)        0.039815
(0, 133)        0.750792
(0, 140)        0.081406
(0, 149)        0.183246
(0, 151)        0.007488
                  ...   
(988, 989)      0.011441
(988, 996)      0.011456
(988, 997)      0.074875
(988, 1000)     0.011432
(988, 1002)     0.011456
(989, 994)      0.011451
(989, 996)      0.011451
(989, 1003)     0.011451
(989, 1004)     0.011451
